### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [1]:
### Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key=os.getenv("OPENAI_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")


In [2]:
!pip install langchain_groq

In [3]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002AD02368E50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002AD0236A2C0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
!pip install langchain_core

In [5]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)

In [6]:
result

AIMessage(content='Bonjour \n\nComment allez-vous ? \n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 21, 'total_tokens': 34, 'completion_time': 0.023636364, 'prompt_time': 0.002145835, 'queue_time': 0.24639673499999998, 'total_time': 0.025782199}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--947cb67b-f116-4564-b3da-8e4a03898c69-0', usage_metadata={'input_tokens': 21, 'output_tokens': 13, 'total_tokens': 34})

In [7]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'Bonjour \n\nComment allez-vous ? \n\n'

In [13]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)


'Here are a few ways to say "Hello, how are you?" in French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Salut, ça va ?** (Informal)\n* **Coucou, comment vas-tu ?** (Very informal, used with friends or family)\n\n\n\nLet me know if you\'d like more variations!\n'

In [19]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Translate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [
        ("system",generic_template),
        ("user","{text}")
    ]
)



In [15]:
result=prompt.invoke({"language":"Korean","text":"Hello"})

In [16]:
result.to_messages()

[SystemMessage(content='Tranaslate the following into Korean:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [18]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"Korean","text":"Hello"})

'안녕하세요 (Annyeonghaseyo) \n'

In [25]:
!pip install streamlit